# Gradio Demo: video_identity

In [5]:
!pip install -q gradio transformers onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 11.9 MB/s eta 0:00:00


In [2]:
# Downloading files from the demo repo
import os
os.mkdir('video')
!wget -q -O video/video_sample.mp4 https://github.com/gradio-app/gradio/raw/main/demo/video_identity/video/video_sample.mp4

In [18]:
import gradio as gr
import onnxruntime
from transformers import AutoTokenizer
import torch
import os
import subprocess
import torchaudio
from transformers import pipeline
### --- Audio/Video to txt ---###
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition", 
                    model="openai/whisper-base.en",
                    chunk_length_s=30, device=device)

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
     

def video_identity(video):
    transcription = pipe(video)["text"]
    return transcription

def summary(text):
    text = text.split('.')
    max_chunk = 500
    current_chunk = 0
    chunks = []
    
    
    for t in text:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(t.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(t.split(' '))
            else:
                current_chunk += 1
                chunks.append(t.split(' '))
        else:
          chunks.append(t.split(' '))
    
    for chunk in range(len(chunks)):
        chunks[chunk] =' '.join(chunks[chunk])
    
    summ = summarizer(chunks,max_length = 100)

    return summ
    
    



token  = AutoTokenizer.from_pretrained('distilroberta-base')

inf_session = onnxruntime.InferenceSession('/content/drive/MyDrive/classifier1-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

classes = ['Art', 'Astrology', 'Biology', 'Chemistry', 'Economics', 'History', 'Literature', 'Philosophy', 'Physics', 'Politics', 'Psychology', 'Sociology']

def classify(vid):
    full_text = video_identity(vid)
    sum = summary(full_text)[0]['summary_text']

    
    input_ids = token(sum)['input_ids'][:512]
    logits = inf_session.run([output_name],{input_name : [input_ids]})[0]
    logits = torch.FloatTensor(logits)
    probs = torch.sigmoid(logits)[0]
    return full_text, sum, dict(zip(classes,map(float,probs)))

# label = gr.outputs.Label(num_top_classes=5)
iface = gr.Interface(fn=classify,
                     inputs=gr.inputs.Audio(source="upload", type="filepath"),
                     outputs = ['text','text',gr.outputs.Label(num_top_classes=3)])

iface.launch()


/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
